In [1]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import os
import dash_mantine_components as dmc
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_table
import dask.dataframe as dd
import re


/Users/vishalsehgal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/hd/p1cpt0jj6z5_ngrvf3vvz4tr0000gn/T/ipykernel_5633/93960180.py:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/hd/p1cpt0jj6z5_ngrvf3vvz4tr0000gn/T/ipykernel_5633/93960180.py:11: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/hd/p1cpt0jj6z5_ngrvf3vvz4tr0000gn/T/ipykernel_5633/93960180.py:13: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table

Main Feature
1. Players performance coomparision : Compare players from different clubs when they face each other in their national teams during the World Cup.
2. Team performance comparision : Analyze team performance and how well players know each other's strengths and weaknesses.


Steps:
1. Integrate Match Data:    Use the Fifa_Worldcup_2022_Groups.csv for match data.
2. Filter Players Based on Match Selection:     Allow users to filter players by match, country, and club.
3. Visual Comparison:   Display a visual comparison of player performance.

# Get Paths and Data

In [2]:
# Load data files
player_data_file = 'assets/data/FIFA22_official_data.csv'
match_data_file = 'assets/data/Matchdata.csv'
group_data_file = 'assets/data/Fifa_Worldcup_2022_Groups.csv'
group_df = pd.read_csv(group_data_file)
player_df = pd.read_csv(player_data_file)
match_df = pd.read_csv(match_data_file)

In [3]:
import pandas as pd
import os

# Paths to the CSV files
player_stats_path = 'assets/Data/FIFA World Cup 2022 Player Data/player_stats.csv'
player_misc_path = 'assets/Data/FIFA World Cup 2022 Player Data/player_misc.csv'

# Load the CSV files into DataFrames
player_stats_df = pd.read_csv(player_stats_path, encoding='utf-8')
player_misc_df = pd.read_csv(player_misc_path, encoding='utf-8')

# Strip whitespace from headers
player_stats_df.columns = player_stats_df.columns.str.strip()
player_misc_df.columns = player_misc_df.columns.str.strip()

# Strip whitespace from string data
for col in player_stats_df.select_dtypes(['object']).columns:
    player_stats_df[col] = player_stats_df[col].str.strip()

for col in player_misc_df.select_dtypes(['object']).columns:
    player_misc_df[col] = player_misc_df[col].str.strip()

# Merge the DataFrames on the 'player' column with suffixes
player_misc_df = pd.merge(player_stats_df, player_misc_df, on='player', how='outer', suffixes=('_stats', '_misc'))

# Function to remove duplicate columns based on content
def remove_duplicate_columns(df):
    # Transpose, drop duplicates, and transpose back
    df = df.T.drop_duplicates().T
    return df

# Apply the function to remove duplicate columns
player_misc_df = remove_duplicate_columns(player_misc_df)

# Remove suffixes from the column names
player_misc_df.columns = player_misc_df.columns.str.replace(r'(_stats|_misc)$', '', regex=True)
player_misc_df.dropna(subset=['club'], inplace=True)
clubs = player_misc_df['club'].unique()

In [4]:
# Merging data
merged_df = match_df.merge(group_df, left_on='home_team', right_on='Country_Name', how='left')
merged_df = merged_df.merge(group_df, left_on='away_team', right_on='Country_Name', how='left', suffixes=('_home', '_away'))
merged_with_home_players_ddf = dd.from_pandas(merged_df, npartitions=4)
player_ddf = dd.from_pandas(player_df, npartitions=4)
final_merged_ddf = dd.merge(
    merged_with_home_players_ddf,
    player_ddf,
    left_on='Country_Name_away',
    right_on='Nationality',
    how='left',
    suffixes=('_home', '_away')
)
final_merged_df = final_merged_ddf.compute()

In [5]:
# Player stats
player_stats = player_df[['Name', 'Nationality', 'Overall', 'Club', 
                          'Crossing', 'Finishing', 'HeadingAccuracy', 
                          'ShortPassing', 'Volleys', 'Dribbling', 
                          'Curve', 'FKAccuracy', 'LongPassing', 
                          'BallControl', 'Acceleration', 'SprintSpeed', 
                          'Agility', 'Reactions', 'Balance', 'ShotPower', 
                          'Jumping', 'Stamina', 'Strength', 'LongShots', 
                          'Aggression', 'Interceptions', 'Positioning', 
                          'Vision', 'Penalties', 'Composure']]

#unique_matches = match_df.apply(lambda x: f"{x['home_team']} vs {x['away_team']}", axis=1).unique()
#unique_players = player_stats['Name'].unique()


In [6]:
def calculate_team_records(match_df):
    team_records = {}

    for index, row in match_df.iterrows():
        home_team = row['home_team']
        away_team = row['away_team']
        score = row['score']
        
        # Clean the score to replace non-ASCII dash with ASCII dash and remove any extra information
        score = re.sub(r'[^0-9–-]', '', score).replace('–', '-')
        
        try:
            home_goals, away_goals = map(int, score.split('-'))
        except ValueError:
            print(f"Skipping invalid score: {score}")
            continue
        
        if home_team not in team_records:
            team_records[home_team] = {}
        if away_team not in team_records:
            team_records[away_team] = {}

        if away_team not in team_records[home_team]:
            team_records[home_team][away_team] = []
        if home_team not in team_records[away_team]:
            team_records[away_team][home_team] = []

        match_detail = {
            'home_team': home_team,
            'away_team': away_team,
            'home_goals': home_goals,
            'away_goals': away_goals
        }
        
        team_records[home_team][away_team].append(match_detail)
        team_records[away_team][home_team].append(match_detail)

    return team_records

team_records = calculate_team_records(match_df)


In [7]:
# Initialize the Dash app
app = JupyterDash(__name__,
        external_stylesheets=[
            # include google fonts
            "https://fonts.googleapis.com/css2?family=Inter:wght@100;200;300;400;500;900&display=swap"
        ],
)

/Users/vishalsehgal/Library/Python/3.9/lib/python/site-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [8]:
app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Players', children=[
            # Header Section
            html.Header([
                html.Img(src=app.get_asset_url('Worldcuplogo.png'), className='header-logo'),  # Logo
                html.Div([
                    html.H1('FIFA SHOWDOWN', className='header-title'),
                    html.H2('Comparing Player Performance Across Teams', className='header-content')
                ], className='header-content'),
            ], className='header'),

            # Dropdowns for Player Selection
            html.Div([
                dcc.Dropdown(
                    id='player1-dropdown',
                    options=[{'label': player, 'value': player} for player in player_stats['Name'].unique()],
                    value=None,
                    placeholder="Select Player 1",
                    className='dropdown'
                ),
                html.Div([
                    dcc.Checklist(
                        id='same-club-checkbox',
                        options=[{'label': 'Choose Players from the Same Club', 'value': 'same_club'}],
                        value=[],
                        className='checkbox'
                    ),
                    dcc.Checklist(
                        id='same-position-checkbox',
                        options=[{'label': 'Choose Players from the Same Position', 'value': 'same_position'}],
                        value=[],
                        className='checkbox'
                    ),
                    dcc.Checklist(
                        id='same-country-checkbox',
                        options=[{'label': 'Choose Players from the Same Country', 'value': 'same_country'}],
                        value=[],
                        className='checkbox'
                    ),
                ], className='checkbox-container'),
                html.Br(),
                dcc.Dropdown(
                    id='player2-dropdown',
                    options=[],
                    value=None,
                    placeholder="Select Player 2",
                    className='dropdown'
                ),
            ], className='dropdown-container'),

            # Container for the card components
            html.Div(
                className="container",
                children=[
                    # Left Segment for Player 1
                    html.Div(
                        className="segment left_segment",
                        children=[
                            html.Div(
                                className='left_card_div',
                                children=[
                                    html.Div(
                                        className='cardcomponent',
                                        children=[
                                            html.Div(id='player1-header', className='header'),  # Player 1 Header
                                            html.Div(
                                                className='content',
                                                children=[
                                                    html.Div(
                                                        className='picture-container',
                                                        style={'display': 'flex', 'alignItems': 'center'},
                                                        children=[
                                                            html.Img(id='player1-image', className='player-image', src='https://cdn.sofifa.net/players/020/801/22_60.png'),
                                                            html.Img(id='player1-club-image', className='club-image', src='https://cdn.sofifa.net/teams/11/30.png'),
                                                            html.Img(id='player1-flag-image', className='flag-image', src='https://cdn.sofifa.net/flags/pt.png')
                                                        ]
                                                    ),
                                                    dash_table.DataTable(
                                                        id='stats-table1',
                                                        columns=[
                                                            {"name": "Stat", "id": "stat"},
                                                            {"name": "Value", "id": "value"}
                                                        ],
                                                        data=[],
                                                        style_table={'overflowX': 'auto'},
                                                        style_header={'backgroundColor': '#f1f1f1'},
                                                        style_cell={'textAlign': 'left', 'padding': '8px', 'border': '1px solid #ddd'},
                                                        style_as_list_view=True
                                                    )
                                                ]
                                            )
                                        ]
                                    )
                                ]
                            )
                        ]
                    ),
                    # Middle Segment for Graph
                    html.Div(
                        className="segment middle_segment",
                        children=[
                            html.Div(id='team-records', style={'padding': '10px', 'fontSize': '16px'}),
                            dcc.Graph(id='radar-plot'),
                            dcc.Graph(id='scatter-plot'),
                            html.Div(
                                className='dropdown-row',
                                children=[
                                    dcc.Dropdown(
                                        id='scatter-x-axis',
                                        options=[{'label': player_stats.columns[2], 'value': player_stats.columns[2]}] + [{'label': stat, 'value': stat} for stat in player_stats.columns[4:]],
                                        value='Overall',
                                        placeholder="Select X-axis",
                                        className='dropdown'
                                    ),
                                    dcc.Dropdown(
                                        id='scatter-y-axis',
                                        options=[{'label': player_stats.columns[2], 'value': player_stats.columns[2]}] + [{'label': stat, 'value': stat} for stat in player_stats.columns[4:]],
                                        value='Reactions',
                                        placeholder="Select Y-axis",
                                        className='dropdown'
                                    ),
                                ],
                            ),
                            html.Div(
                                className='graph-row',
                                children=[
                                    dcc.Graph(id='boxplot'),   
                                ]
                            ),
                            html.Div(
                                className='dropdown-row',
                                children=[
                                    dcc.Dropdown(
                                        id='stat-dropdown',
                                        options=[{'label': player_stats.columns[2], 'value': player_stats.columns[2]}] + [{'label': stat, 'value': stat} for stat in player_stats.columns[4:]],
                                        value=player_df.columns[2],  # Default value
                                        placeholder="Select Stat",
                                        className='dropdown'
                                    ),
                                ],
                            ),
                        ]
                    ),

                    # Right Segment for Player 2
                    html.Div(
                        className="segment right_segment",
                        children=[
                            html.Div(
                                className='right_card_div',
                                children=[
                                    html.Div(
                                        className='cardcomponent',
                                        children=[
                                            html.Div(id='player2-header', className='header'),  # Player 2 Header
                                            html.Div(
                                                className='content',
                                                children=[
                                                    html.Div(
                                                        className='picture-container',
                                                        style={'display': 'flex', 'alignItems': 'center'},
                                                        children=[
                                                            html.Img(id='player2-image', className='player-image', src=''),
                                                            html.Img(id='player2-club-image', className='club-image', src=''),
                                                            html.Img(id='player2-flag-image', className='flag-image', src='')
                                                        ]
                                                    ),
                                                    dash_table.DataTable(
                                                        id='stats-table2',
                                                        columns=[
                                                            {"name": "Stat", "id": "stat"},
                                                            {"name": "Value", "id": "value"}
                                                        ],
                                                        data=[],
                                                        style_table={'overflowX': 'auto'},
                                                        style_header={'backgroundColor': '#f1f1f1'},
                                                        style_cell={'textAlign': 'left', 'padding': '8px', 'border': '1px solid #ddd'},
                                                        style_as_list_view=True
                                                    )
                                                ]
                                            )
                                        ]
                                    )
                                ]
                            )
                        ]
                    )
                ]
            )
        ]),
        dcc.Tab(label='Clubs', children=[
            html.Div([
                dcc.Dropdown(
                    id='club-dropdown',
                    options=[{'label': club, 'value': club} for club in clubs],
                    value=[],
                    placeholder="Select Clubs",
                    className='club-dropdown',
                    multi=True  # Enable multiple selections
                ),
                html.Div([
                    dcc.Dropdown(
                        id='feature-dropdown',
                        options=[{'label': feature, 'value': feature} for feature in player_misc_df.columns[8:]],
                        value=None,
                        placeholder="Select Feature",
                        className='feature-dropdown'
                    ),
                ], className='dropdown-container'),
                html.Div([
                    dcc.Graph(id='club-bar-chart'),
                ], className='graph-container'),
            ], className='club-dropdown-container'),
        ]),
    ]),
])

In [9]:
# Define the callback to update player dropdowns based on selected player and checkboxes
@app.callback(
    Output('player2-dropdown', 'options'),
    [Input('player1-dropdown', 'value'),
     Input('same-club-checkbox', 'value'),
     Input('same-position-checkbox', 'value'),
     Input('same-country-checkbox', 'value')]
)
def update_player2_dropdown(player1, same_club, same_position, same_country):
    if player1 is None:
        return [{'label': player, 'value': player} for player in player_df['Name'].unique()]

    player1_data = player_df[player_df['Name'] == player1].iloc[0]
    filtered_players = player_df.copy()

    if 'same_club' in same_club:
        filtered_players = filtered_players[filtered_players['Club'] == player1_data['Club']]

    if 'same_position' in same_position:
        filtered_players = filtered_players[filtered_players['Position'] == player1_data['Position']]

    if 'same_country' in same_country:
        filtered_players = filtered_players[filtered_players['Nationality'] == player1_data['Nationality']]

    # Exclude the player selected in player1-dropdown
    filtered_players = filtered_players[filtered_players['Name'] != player1]

    return [{'label': player, 'value': player} for player in filtered_players['Name'].unique()]

# Define the callback to update the radar plot
@app.callback(
    Output('radar-plot', 'figure'),
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_radarplot(player1, player2):
    if player1 is None or player2 is None:
        return go.Figure()

    player1_stats = player_stats[player_stats['Name'] == player1].iloc[0]
    player2_stats = player_stats[player_stats['Name'] == player2].iloc[0]

    categories = player1_stats.index[2:-1]  # Adjust index to match your data structure, excluding the Photo column
    player1_values = player1_stats[2:-1].values
    player2_values = player2_stats[2:-1].values

    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=player1_values,
        theta=categories,
        fill='toself',
        name=player1
    ))

    fig.add_trace(go.Scatterpolar(
        r=player2_values,
        theta=categories,
        fill='toself',
        name=player2
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        showlegend=True,
        title=f'Comparison: {player1} vs {player2}'
    )

    return fig

# Define the callback to update the scatter plot to compare player with all the players in the dataset
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value'),
     Input('scatter-x-axis', 'value'),
     Input('scatter-y-axis', 'value')]
)
def update_scatterplot(player1, player2, xaxis, yaxis):
    if player1 is None or player2 is None or xaxis is None or yaxis is None:
        return go.Figure()

    player1_stats = player_stats[player_stats['Name'] == player1].iloc[0]
    player2_stats = player_stats[player_stats['Name'] == player2].iloc[0]

    player1_x = player1_stats[xaxis]
    player1_y = player1_stats[yaxis]
    player2_x = player2_stats[xaxis]
    player2_y = player2_stats[yaxis]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=player_stats[xaxis],
        y=player_stats[yaxis],
        mode='markers',
        marker=dict(
            size=12,
            color=player_stats[xaxis],
            colorscale='Viridis',
            showscale=True
        ),
        text=player_stats['Name'],
        name='All Players'
    ))
    fig.add_trace(go.Scatter(
        x=[player1_x],
        y=[player1_y],
        mode='markers',
        marker=dict(
            size=12,
            color='red'
        ),
        text=[player1],
        name=player1
    ))

    fig.add_trace(go.Scatter(
        x=[player2_x],
        y=[player2_y],
        mode='markers',
        marker=dict(
            size=12,
            color='blue'
        ),
        text=[player2],
        name=player2
    ))

    fig.update_layout(
        title='Player Comparison',
        xaxis_title=xaxis,
        yaxis_title=yaxis
    )

    return fig

# Define the callback to update the boxplot
@app.callback(
    Output('boxplot', 'figure'),
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value'),
     Input('stat-dropdown', 'value')]
)
def update_boxplots(player1, player2, selected_stat):
    if player1 is None or player2 is None or selected_stat is None:
        return go.Figure()

    # Filter stats for the selected players
    player1_stats = player_df[player_df['Name'] == player1].iloc[0]
    player2_stats = player_df[player_df['Name'] == player2].iloc[0]

    # Ensure the 'Position' column name matches your CSV
    player1_position = player1_stats['Position']
    player2_position = player2_stats['Position']

    # Filter stats for players in the same positions
    similar_position1_stats = player_df[player_df['Position'] == player1_position]
    similar_position2_stats = player_df[player_df['Position'] == player2_position]

    # Initialize the figure
    fig = go.Figure()

    if player1_position == player2_position:
        # If both players are from the same position, create a single box plot
        fig.add_trace(go.Box(
            y=similar_position1_stats[selected_stat],
            x=[player1_position] * len(similar_position1_stats),
            name=f'{player1_position} Players',
            marker_color='lightgrey',
            boxmean='sd',  # Display standard deviation
            jitter=0.3,
            pointpos=-1.8,
            boxpoints='all'
        ))

        # Add scatter plot for the selected players
        fig.add_trace(go.Scatter(
            y=[player1_stats[selected_stat]],
            x=[player1_position],
            name=f'{player1} - {selected_stat}',
            mode='markers',
            marker=dict(color='red', size=10, symbol='circle')
        ))

        fig.add_trace(go.Scatter(
            y=[player2_stats[selected_stat]],
            x=[player1_position],
            name=f'{player2} - {selected_stat}',
            mode='markers',
            marker=dict(color='blue', size=10, symbol='circle')
        ))

    else:
        # If the players are from different positions, create separate box plots for each position

        # Add box plot for the similar position players of player1
        fig.add_trace(go.Box(
            y=similar_position1_stats[selected_stat],
            x=[player1_position] * len(similar_position1_stats),
            name=f'{player1_position} Players',
            marker_color='lightgrey',
            boxmean='sd',  # Display standard deviation
            jitter=0.3,
            pointpos=-1.8,
            boxpoints='all'
        ))

        # Add scatter plot for the selected player1
        fig.add_trace(go.Scatter(
            y=[player1_stats[selected_stat]],
            x=[player1_position],
            name=f'{player1} - {selected_stat}',
            mode='markers',
            marker=dict(color='red', size=10, symbol='circle')
        ))

        # Add box plot for the similar position players of player2
        fig.add_trace(go.Box(
            y=similar_position2_stats[selected_stat],
            x=[player2_position] * len(similar_position2_stats),
            name=f'{player2_position} Players',
            marker_color='lightgrey',
            boxmean='sd',  # Display standard deviation
            jitter=0.3,
            pointpos=-1.8,
            boxpoints='all'
        ))

        # Add scatter plot for the selected player2
        fig.add_trace(go.Scatter(
            y=[player2_stats[selected_stat]],
            x=[player2_position],
            name=f'{player2} - {selected_stat}',
            mode='markers',
            marker=dict(color='blue', size=10, symbol='circle')
        ))

    # Update layout
    fig.update_layout(
        title=f'{selected_stat} Distribution for {player1} and {player2} and Their Respective Position Players',
        xaxis_title='Position',
        yaxis_title=selected_stat,
        boxmode='group'
    )

    return fig

# Define the callback to update the header text based on player1 and player2 selection
@app.callback(
    [Output('player1-header', 'children'),
     Output('player2-header', 'children')],
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_headers(player1, player2):
    player1_header = f"{player1}'s Stats" if player1 else "Player 1 Stats"
    player2_header = f"{player2}'s Stats" if player2 else "Player 2 Stats"
    return player1_header, player2_header

# Define the callback to update DataTables based on player1 and player2 selection
@app.callback(
    [Output('stats-table1', 'data'),
     Output('stats-table2', 'data')],
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_tables(player1, player2):
    player1_data = []
    player2_data = []

    if player1:
        player1_stats = player_stats[player_stats['Name'] == player1].iloc[0]
        player1_data = [{"stat": stat, "value": player1_stats[stat]} for stat in player1_stats.index[2:-1]]
    
    if player2:
        player2_stats = player_stats[player_stats['Name'] == player2].iloc[0]
        player2_data = [{"stat": stat, "value": player2_stats[stat]} for stat in player2_stats.index[2:-1]]

    return player1_data, player2_data

# Define the callback to update player images based on selection
@app.callback(
    [Output('player1-image', 'src'),
     Output('player1-club-image', 'src'),
     Output('player1-flag-image', 'src'),
     Output('player2-image', 'src'),
     Output('player2-club-image', 'src'),
     Output('player2-flag-image', 'src')],
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_images(player1, player2):
    # Default high-resolution images
    default_image1 = 'https://cdn.sofifa.net/players/158/023/22_240.png'
    default_image2 = 'https://cdn.sofifa.net/players/020/801/22_240.png'
    default_club1_image = 'https://cdn.sofifa.net/teams/73/240.png'
    default_flag1_image = 'https://cdn.sofifa.net/flags/ar@3x.png'
    default_club2_image = 'https://cdn.sofifa.net/teams/11/240.png'
    default_flag2_image = 'https://cdn.sofifa.net/flags/pt@3x.png'

    # Set default images
    player1_image = default_image1
    player2_image = default_image2
    player1_club_image = default_club1_image
    player2_club_image = default_club2_image
    player1_flag_image = default_flag1_image
    player2_flag_image = default_flag2_image

    # Update images if player1 is selected
    if player1:
        player1_image_url = player_df[player_df['Name'] == player1]['Photo'].values[0]
        player1_club_url = player_df[player_df['Name'] == player1]['Club Logo'].values[0]
        player1_flag_url = player_df[player_df['Name'] == player1]['Flag'].values[0]
        if pd.notna(player1_image_url):
            player1_image = player1_image_url.replace('22_60.png', '22_240.png')
        if pd.notna(player1_club_url):
            player1_club_image = player1_club_url.replace('/30.png', '/240.png')
        if pd.notna(player1_flag_url):
            player1_flag_image = player1_flag_url.replace('.png', '@3x.png')

    # Update images if player2 is selected
    if player2:
        player2_image_url = player_df[player_df['Name'] == player2]['Photo'].values[0]
        player2_club_url = player_df[player_df['Name'] == player2]['Club Logo'].values[0]
        player2_flag_url = player_df[player_df['Name'] == player2]['Flag'].values[0]
        if pd.notna(player2_image_url):
            player2_image = player2_image_url.replace('22_60.png', '22_240.png')
        if pd.notna(player2_club_url):
            player2_club_image = player2_club_url.replace('/30.png', '/240.png')
        if pd.notna(player2_flag_url):
            player2_flag_image = player2_flag_url.replace('.png', '@3x.png')

    # Return the updated image URLs
    return player1_image, player1_club_image, player1_flag_image, player2_image, player2_club_image, player2_flag_image

# Define the callback to update Left card and right card values to red and green based on higher value of the two players with a exception of overall and club
@app.callback(
    [Output('stats-table1', 'style_data_conditional'),
     Output('stats-table2', 'style_data_conditional')],
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_table_colors(player1, player2):
    if player1 is None or player2 is None:
        return [], []

    player1_stats = player_stats[player_stats['Name'] == player1].iloc[0]
    player2_stats = player_stats[player_stats['Name'] == player2].iloc[0]

    style_data_conditional1 = []
    style_data_conditional2 = []

    for stat in player1_stats.index[2:-1]:
        if stat in ['Overall', 'Club']:
            continue

        player1_value = player1_stats[stat]
        player2_value = player2_stats[stat]

        if player1_value > player2_value:
            style_data_conditional1.append({
                'if': {'filter_query': f'{{stat}} = "{stat}"'},
                'backgroundColor': '#d1f7cf'
            })
            style_data_conditional2.append({
                'if': {'filter_query': f'{{stat}} = "{stat}"'},
                'backgroundColor': '#f7d1d1'
            })
        elif player1_value < player2_value:
            style_data_conditional1.append({
                'if': {'filter_query': f'{{stat}} = "{stat}"'},
                'backgroundColor': '#f7d1d1'
            })
            style_data_conditional2.append({
                'if': {'filter_query': f'{{stat}} = "{stat}"'},
                'backgroundColor': '#d1f7cf'
            })

    return style_data_conditional1, style_data_conditional2



# callback to update the scores of the match based of the players nationalities
@app.callback(
    Output('team-records', 'children'),
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_team_records(player1, player2):
    if player1 and player2:
        # Retrieve the national teams (nationality) of the selected players
        team1 = player_df.loc[player_df['Name'] == player1, 'Nationality'].values[0]
        team2 = player_df.loc[player_df['Name'] == player2, 'Nationality'].values[0]

        print(f"Selected teams: {team1} vs {team2}")  # Debug statement

        # Check if the selected teams have a record against each other
        if team1 in team_records and team2 in team_records[team1]:
            matches = team_records[team1][team2]
            print(f"Matches found: {matches}")  # Debug statement

            match_list = [
                html.Div(f"{match['home_team']} {match['home_goals']}-{match['away_goals']} {match['away_team']}")
                for match in matches
            ]
            return match_list
        else:
            return f'No matches found between {team1} and {team2} in the World Cup.'

    return 'Select both players to see the match history.'

# call back for clubs dropdown in club tab
@app.callback(
    Output('club-dropdown', 'options'),
    [Input('club-dropdown', 'value')]
)
def update_club_dropdown(selected_club):
    return [{'label': club, 'value': club} for club in player_misc_df['club'].unique()]

# callback for club bar chart to show overall rating of whole club based on drop downs
@app.callback(
    Output('club-bar-chart', 'figure'),
    [Input('club-dropdown', 'value'),
     Input('feature-dropdown', 'value')]
)
def update_scatter_plot(selected_clubs, selected_feature):
    if not selected_clubs or selected_feature is None:
        return go.Figure()

    # Ensure selected_clubs is a list
    if isinstance(selected_clubs, str):
        selected_clubs = [selected_clubs]

    # Calculate the combined feature value for each selected club
    club_values = []
    for club in selected_clubs:
        filtered_players = player_misc_df[player_misc_df['club'] == club]
        combined_feature_value = filtered_players[selected_feature].sum()
        club_values.append(combined_feature_value)

    # Create the scatter plot
    fig = go.Figure()

    # Adding points for the selected clubs with their combined feature values
    fig.add_trace(go.Scatter(
        x=selected_clubs,
        y=club_values,
        mode='markers',
        marker=dict(size=10, color='blue'),
        name=selected_feature
    ))

    # Update the layout of the scatter plot
    fig.update_layout(
        title=f'Combined {selected_feature} of Players in Selected Clubs',
        xaxis_title='Club',
        yaxis_title=selected_feature,
        xaxis_tickangle=-45
    )

    return fig



if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
